In [13]:
from sklearn.metrics import classification_report
import torch
import pandas as pd
from tqdm import tqdm
import transformers
from transformers import AutoModelForMaskedLM, AutoModelForSequenceClassification,AutoTokenizer, AutoModel, AutoConfig, utils
from transformers import pipeline

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf

RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
initialization of _pywrap_checkpoint_reader raised unreported exception

In [ ]:
pip install transformers

In [3]:
#!pip install transformers
#!pip show transformers
#!pip install torch torchvision
#!pip install -U scikit-learn
#!pip install openpyxl

In [4]:
orig_sentence_file =  "../Project2_Gaze4Hate/gaze4hate_sentences_to_annotate.xlsx"
orig_df = pd.read_excel(open(orig_sentence_file, 'rb'), sheet_name='stimuli') 

## remove the 4 familiarization sentences (last four)
orig_df= orig_df.loc[orig_df['Cond_remark']!='fam']

In [5]:
orig_df.Cond_remark.unique()

array(['hate_w_high', 'no_hate_w', 'hate_w_med', 'hate_m', 'neutral_w',
       'pos_w'], dtype=object)

In [6]:
orig_df

,Sno,Condition,Cond_remark,ID,processed_assertion,H_P_mw,H_N_mw
0,1,1,hate_w_high,27,Frauen haben einen signifikant geringeren Durc...,"[5,6]",[1]
1,2,1,hate_w_high,26,Frauen sind psychisch nicht für komplexe Arbei...,[4],[1]
2,3,1,hate_w_high,16,Frauen haben im statistischen Mittel ein klein...,"[7,8]",[1]
3,4,1,hate_w_high,14,Frauenförderung hat fast nur Nachteile für die...,[5],[1]
4,5,1,hate_w_high,2,Frauen sind im Durchschnitt dümmer als Männer.,[5],"[1,7]"
...,...,...,...,...,...,...,...
85,89,3,pos_w,28,Gleichstellung der Geschlechter ist ein Traum ...,"[6,7,8,9]",[99]
86,90,3,pos_w,32,Frauen in führenden Positionen haben häufig st...,[7],[99]
87,91,3,pos_w,1,Frauen gehören in die Regierung.,[5],[99]
88,92,3,pos_w,12,Frauen sollten ihren Traumberuf ausüben.,[4],[99]


In [7]:
### initial six categories can be categorized into two as hate versus nohate
orig_df.loc[orig_df['Cond_remark']=='hate_w_high', 'label']= 1 # hate
orig_df.loc[orig_df['Cond_remark']=='no_hate_w', 'label']= 0 # nohate
orig_df.loc[orig_df['Cond_remark']=='hate_w_med', 'label']= 1 # hate
orig_df.loc[orig_df['Cond_remark']=='hate_m', 'label']= 1 # hate
orig_df.loc[orig_df['Cond_remark']=='neutral_w', 'label']= 0 # nohate
orig_df.loc[orig_df['Cond_remark']=='pos_w', 'label']= 0 # nohate

In [8]:
def predict(model, encodings):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    masks = torch.tensor([[int(e > 0) for e in encodings]], device=device)
    input = torch.tensor([encodings], dtype=torch.long, device=device)
    # get model output
    output = model(input, masks)
    pred_id = torch.argmax(output[0], dim=1).item()
    #pred_label = model.config.id2label[pred_id]

    return pred_id

In [9]:

def main(orig_df, bert_type):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    if bert_type=='base':
        tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-german-uncased")   
        model = AutoModelForSequenceClassification.from_pretrained('dbmdz/bert-base-german-uncased', output_hidden_states=True, return_dict=True, output_attentions=True).to(device)

    if bert_type=="germeval18" :   
        tokenizer = AutoTokenizer.from_pretrained("deepset/bert-base-german-cased-hatespeech-GermEval18Coarse")
        model = AutoModelForSequenceClassification.from_pretrained("deepset/bert-base-german-cased-hatespeech-GermEval18Coarse",output_hidden_states=True, return_dict=True, output_attentions=True).to(device)   
    

    # get input

    texts = orig_df['processed_assertion']
    
    # encode input
    encodings = [tokenizer.encode(t) for t in texts]
    
    # predict testset
    predictions = [predict(model, e) for e in tqdm(encodings)]
    
    #print(predictions)
    # save preds
    orig_df[bert_type+'_PREDS'] = predictions
    orig_df.to_csv('gaze4hate_sentence_bert_predictions_'+ bert_type+'.csv', index=False)
    
    labels = orig_df['label']
    #print(labels)

     # evaluate predictions
    print(classification_report(labels, predictions))

In [10]:
main(orig_df, 'base')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-german-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  0%|          | 0/90 [00:00<?, ?it/s]

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
tensor([[  102,  1648,   423,   366, 18014, 28385, 22226, 24187,  5926,  8791,
         30938,   552,   103]])


  3%|▎         | 3/90 [00:00<00:19,  4.43it/s]

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
tensor([[  102,  1648,   321,  3825,   187,   237,  9586, 18334,  1029,  5056,
           552,   103]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
tensor([[  102,  1648,   423,   197, 12597,  1085,   139,  8689, 30941,  1954,
          2789,   552,   103]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
tensor([[  102,  1648,  6344,   292,  1932,   435, 21736,  9586,   125,  2247,
           552,   103]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
tensor([[  102,  1648,   321,   197,  5541, 12787,   105,   250,   946,   105,
           552,   103]])


  8%|▊         | 7/90 [00:01<00:08,  9.73it/s]

...

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
tensor([[ 102,  125, 1603, 1744, 5892,  223, 5561,  207, 1337, 4344,  209,  552,
          103]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
tensor([[  102,  1648,  2152,   237,   167,  3637,  2647,   143,   235,   271,
           597,  4478, 18660,   334, 30938,   552,   103]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
tensor([[  102, 21629,  1165,   321, 24026,  4968, 19634,   329, 13018,  5926,
          3745, 30937,   552,   103]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
tensor([[  102, 21078,   124,  4826,   281,  6403,   806,  1648,  4826,   281,
          2823,   552,   103]])


 96%|█████████▌| 86/90 [00:05<00:00, 17.64it/s]

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
tensor([[  102,   238,  6727,  7485, 12452,   142,   127,  2679,   207,   167,
         28081,   552,   103]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
tensor([[  102,  1648,   321,  9586,   676, 19916,   832,  6940,   552,   103]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
tensor([[  102, 22747,   127, 17114,   207,   139,  6263,   127,  2247,   552,
           103]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
tensor([[  102,  1648,   142, 16093,   166, 10792,   423,   792,  4027, 30948,
          6246,   980,  5677,   552,   103]])


100%|██████████| 90/90 [00:05<00:00, 15.15it/s]

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])
tensor([[  102,  1648, 20197,   106,   142,   125,  2119,   552,   103]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
tensor([[  102,  1648,  2152,  1053,  6263, 11435,   216, 10360, 30938,   552,
           103]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
tensor([[  102,  1177,   359,   238,   884,  1575,  1055,   806,   207,   597,
          5557, 30937,   167, 25005,   552,   103]])
              precision    recall  f1-score   support

         0.0       0.56      0.90      0.69        50
         1.0       0.50      0.12      0.20        40

    accuracy                           0.56        90
   macro avg       0.53      0.51      0.45        90
weighted avg       0.53      0.56      0.47        90



In [16]:
main(orig_df, 'germeval18')

100%|██████████| 90/90 [00:05<00:00, 15.73it/s]


              precision    recall  f1-score   support

         0.0       0.61      0.78      0.68        50
         1.0       0.58      0.38      0.45        40

    accuracy                           0.60        90
   macro avg       0.59      0.58      0.57        90
weighted avg       0.59      0.60      0.58        90



In [17]:
## EXERCISE: 
## you can try this on Davidson data
davidson_sentence_file =  "../day1_annotation_interrateragreement/labeled_data.csv"
davidson_df = pd.read_csv(davidson_sentence_file) 
davidson_df

## Note that it has multiclass label:
## class = class label for majority of CF users. 0 - hate speech 1 - offensive language 2 - neither


,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies
